To carry out inference, please follow these steps, you don't actually need any API keys for inference, nor GPU

In [ ]:
import torch
import pandas as pd
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, AutoConfig

hf_repo_id = "Savoxism/phobert_v2.2_multilabel"

tokenizer = AutoTokenizer.from_pretrained(hf_repo_id)

class PhoBERTMultiAspectModel(nn.Module):
    def __init__(self, model_name, num_aspects):
        super(PhoBERTMultiAspectModel, self).__init__()

        self.bert = AutoModel.from_pretrained(model_name, output_hidden_states=True)
        self.config = AutoConfig.from_pretrained(model_name)  
        self.hidden_size = self.bert.config.hidden_size * 4  # Concatenating last 4 layers

        # Create a classifier for each aspect (4 logits per aspect)
        self.aspect_classifiers = nn.ModuleList([
            nn.Linear(self.hidden_size, 4) for _ in range(num_aspects)
        ])

        self.dropout = nn.Dropout(0.2)

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, 
                            token_type_ids=token_type_ids if token_type_ids is not None else None)
        
        # Get the hidden states from the BERT output
        hidden_states = outputs.hidden_states 
        
        # Concatenate last 4 hidden layers
        pooled_output = torch.cat([hidden_states[-i][:, 0, :] for i in range(1, 5)], dim=-1)
        pooled_output = self.dropout(pooled_output)

        # Apply classifiers to each aspect
        aspect_outputs = [classifier(pooled_output) for classifier in self.aspect_classifiers]

        return torch.stack(aspect_outputs, dim=1) # (batch_size, num_aspects, 4)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PhoBERTMultiAspectModel(model_name="vinai/phobert-base", num_aspects=34)
model.load_state_dict(torch.hub.load_state_dict_from_url(f"https://huggingface.co/{hf_repo_id}/resolve/main/pytorch_model.bin", map_location=device))
model.to(device)
model.eval()  

# ✅ Define list of aspect categories
label_keys = [
    "FACILITIES#CLEANLINESS", "FACILITIES#COMFORT", "FACILITIES#DESIGN&FEATURES", "FACILITIES#GENERAL",
    "FACILITIES#MISCELLANEOUS", "FACILITIES#PRICES", "FACILITIES#QUALITY", "FOOD&DRINKS#MISCELLANEOUS",
    "FOOD&DRINKS#PRICES", "FOOD&DRINKS#QUALITY", "FOOD&DRINKS#STYLE&OPTIONS", "HOTEL#CLEANLINESS",
    "HOTEL#COMFORT", "HOTEL#DESIGN&FEATURES", "HOTEL#GENERAL", "HOTEL#MISCELLANEOUS", "HOTEL#PRICES",
    "HOTEL#QUALITY", "LOCATION#GENERAL", "ROOMS#CLEANLINESS", "ROOMS#COMFORT", "ROOMS#DESIGN&FEATURES",
    "ROOMS#GENERAL", "ROOMS#MISCELLANEOUS", "ROOMS#PRICES", "ROOMS#QUALITY", "ROOM_AMENITIES#CLEANLINESS",
    "ROOM_AMENITIES#COMFORT", "ROOM_AMENITIES#DESIGN&FEATURES", "ROOM_AMENITIES#GENERAL",
    "ROOM_AMENITIES#MISCELLANEOUS", "ROOM_AMENITIES#PRICES", "ROOM_AMENITIES#QUALITY", "SERVICE#GENERAL"
]

In [ ]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
    inputs = {key: value.to(device) for key, value in inputs.items()}  

    # Run inference
    with torch.no_grad():
        outputs = model(**inputs)  # Shape: (batch_size, num_aspects, 4)

    # convert logits to class predictions
    predicted_labels = torch.argmax(outputs, dim=-1).cpu().numpy()  # ✅ Ensure output is on CPU

    # convert to list
    predicted_labels = predicted_labels.flatten().tolist()  

    # Sentiment mapping
    sentiment_map = {0: "None", 1: "Positive", 2: "Negative", 3: "Neutral"}
    
    aspect_predictions = {aspect: sentiment_map[int(pred)] for aspect, pred in zip(label_keys, predicted_labels)}
    return aspect_predictions


In [ ]:
example_text = input()
predictions = predict(example_text)

predictions_df = pd.DataFrame(list(predictions.items()), columns=["Aspect", "Sentiment"])
print(predictions_df)